In [1]:
from kaggle_environments import make
import json, math, random, os
from itertools import combinations
import pandas as pd

Loading environment football failed: No module named 'gfootball'


In [24]:
def get_results(step):
    city_tiles = [0, 0]
    units = [0, 0]
    for update in step[0]['observation']['updates']:
        split = update.split(' ')
        if split[0] == 'u':
            units[int(split[2])] += 1
        elif split[0] == 'ct':
            city_tiles[int(split[1])] += 1
            
    if city_tiles[0] > city_tiles[1]:
        winner = 0
    elif city_tiles[0] < city_tiles[1]:
        winner = 1
    elif units[0] > units[1]:
        winner = 0
    elif units[0] < units[1]:
        winner = 1
    else:
        winner = -1 # tie
        
    return(city_tiles[0], units[0], city_tiles[1], units[1], winner)


def play_seed(seed, match_id=100000, debug=False):
    env = make('lux_ai_2021', configuration={'seed': seed, 'loglevel': 2, 'annotations': False}, debug=True)
    match_results = []
    chosen_agents = []
    cols = ['seed', 'match_id', 'bot0', 'city_tiles0', 'units0', 'bot1', 'city_tiles1', 'units1', 'winner']
    bots = ['agent.py', './working_title/agent.py', './kaban/agent_tb.py', './kaban/agent_dr.py', './kaban/agent_rl.py']
    names = ['meta_bot', 'working_title', 'imitation_tb', 'imitation_dr', 'imitation_rl']
    
    algo = None
    
    df_ca_list = []
    # when meta_bot playing, need to determine most played arm
    for (bot0, name0), (bot1, name1) in combinations(zip(bots, names), 2):
        # play match
        print("************************************************************************************************************************")
        print(f"Playing {name0} vs {name1}, seed {seed}, match_id {match_id}")
        steps = env.run([bot0, bot1])
        city_tiles0, units0, city_tiles1, units1, winner = get_results(steps[-1])
        
        if winner == 0:
            winner_name = name0
        elif winner == 1:
            winner_name = name1
        else:
            winner_name = 'tie'        
        
        print("************************************************************************************************************************")
        print(f"In {name0} vs {name1}, {winner_name} wins! {name0} had {city_tiles0} city_tiles and {units0} units while {name1} had {city_tiles1} city_tiles and {units1} units.")
        print("************************************************************************************************************************")
        print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
        match_results.append([seed, match_id, name0, city_tiles0, units0, name1, city_tiles1, units1, winner_name])
        
        
        if 'meta_bot' in [name0, name1]:
            # stores which agent was chosen at each time step by the OL algo
            df_ca_single = pd.read_csv('chosen_agent.log', names=['agent', 'raw_reward', 'scaled_reward'], skiprows=2)
            df_ca_single.insert(0, 'seed', seed)
            df_ca_single.insert(1, 'match_id', match_id)
            df_ca_single.insert(2, 'step', range(df_ca_single.shape[0]))
            df_ca_list.append(df_ca_single)
            
        match_id += 1
        
        # get name of OL algo being used
        if algo is None:
            with open('chosen_agent.log', 'r') as f:
                algo = f.readline().rstrip()
        
    df_mr = pd.DataFrame(match_results, columns=cols)
    df_mr.insert(0, 'algo', algo)
    df_ca = pd.concat(df_ca_list)
    df_ca.insert(0, 'algo', algo)
    return(df_mr, df_ca)
    

def experiments(n, debug=False):
    os.makedirs('seed_experiments', exist_ok=True)
    if 'results.csv' in os.listdir('seed_experiments'):
        df_results_orig = pd.read_csv(os.path.join('seed_experiments', 'results.csv'))
        seeds_played = df_results_orig['seed'].unique().tolist()
        match_id = df_results_orig['match_id'].max() + 1
    else:
        df_results_orig = pd.DataFrame()
        seeds_played = []
        match_id = 100000
    
    if 'chosen_agents.csv' in os.listdir('seed_experiments'):
        df_chosen_agents_orig = pd.read_csv(os.path.join('seed_experiments', 'chosen_agents.csv'))
    else:
        df_chosen_agents_orig = pd.DataFrame()
    
    chosen_agents_list = []
    results_list = []
    
    for i in range(n):
        seed = math.floor(random.random() * 1e9)
        while seed in seeds_played:
            seed = math.floor(random.random() * 1e9)    
        
        results, chosen_agents = play_seed(seed, match_id, debug)
        match_id = results['match_id'].max() + 1
        seeds_played.append(seed)
        results_list.append(results)
        chosen_agents_list.append(chosen_agents)
        
    df_results = pd.concat(results_list)
    df_chosen_agents = pd.concat(chosen_agents_list)
        
    # save results
    df_results = pd.concat([df_results_orig, df_results]).reset_index(drop=True)
    df_chosen_agents = pd.concat([df_chosen_agents_orig, df_chosen_agents])
    df_results.to_csv(os.path.join('seed_experiments', 'results.csv'), index=False).reset_index(drop=True)
    df_chosen_agents.to_csv(os.path.join('seed_experiments', 'chosen_agents.csv'), index=False)
    
    return(df_results, df_chosen_agents)

In [ ]:
results, chosen_agents = experiments(4)


************************************************************************************************************************
Playing meta_bot vs working_title, seed 448413026, match_id 100060
agent.py:106: RuntimeWarning: invalid value encountered in double_scalars
  return(self.c * np.log(self.t ** 2)/(self.t * delta_hat ** 2))
agent.py:106: RuntimeWarning: divide by zero encountered in double_scalars
  return(self.c * np.log(self.t ** 2)/(self.t * delta_hat ** 2))
[WARN] (match_xXFoUg4fE3KW) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 31; cmd: bcity u_9
[WARN] (match_xXFoUg4fE3KW) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 32; cmd: bcity u_9
[WARN] (match_xXFoUg4fE3KW) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uranium: 96; turn 33; cmd: bcity u_9
[WARN] (match_xXFoUg4fE3KW) - Agent 1 tried to build CityTile with insufficient materials wood + coal + uraniu

[WARN] (match_xXFoUg4fE3KW) - turn 56; Unit u_1 collided when trying to move e to (4, 9)
[WARN] (match_xXFoUg4fE3KW) - turn 56; Unit u_10 collided when trying to move n to (3, 9)
[WARN] (match_xXFoUg4fE3KW) - turn 56; Unit u_18 collided when trying to move w to (3, 10)
[WARN] (match_xXFoUg4fE3KW) - turn 56; Unit u_2 collided when trying to move w to (4, 9)
[WARN] (match_xXFoUg4fE3KW) - turn 56; Unit u_13 collided when trying to move e to (7, 13)
[WARN] (match_xXFoUg4fE3KW) - turn 56; Unit u_11 collided when trying to move s to (7, 13)
[WARN] (match_xXFoUg4fE3KW) - turn 57; Unit u_18 collided when trying to move n to (4, 9)
[WARN] (match_xXFoUg4fE3KW) - turn 57; Unit u_2 collided when trying to move w to (4, 9)


In [ ]:
results

In [ ]:
chosen_agents